In [3]:
%matplotlib inline 

import numpy as np
import pandas as pd
from scipy.stats import pearsonr 
import matplotlib.pyplot as plt
import glob
import os
import time
import xarray as xr

# import dask.array as da
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster

import matplotlib.patheffects as PathEffects

### load lidar xarray

In [4]:
fname = '~/Documents/projects/thesis/results/output/compiled_SUPERsnow.nc' #ARS
# fname = '~/Documents/research/sierra/data/compiled_SUPERsnow.nc' #BSU
#~~~~ ds full
ds = xr.open_dataset(fname,  chunks={'time':1,'x':1000,'y':1000})
ds.close()

#~~~~ ds peak
dpeak = ds.isel(time=[0,7,18,30,42,49]) #0,7,22,28,41,49]
dpeak.close()

ds = dpeak
ds.close()

### load terrain xarray

In [13]:
fpath = glob.glob('/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/*.nc')
fpath


['/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/hillshade.nc',
 '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/roughness.nc',
 '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/slope.nc',
 '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/surface_classification.nc',
 '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/aspect.nc']

In [15]:
# ds_terr=xr.open_mfdataset(fpath, concat_dim='space') #, chunks={'x':1000, 'y':1000}, parallel=True) # .rename({'Band1':'snow'}).drop('transverse_mercator') combine='nested',
# ds_terr

hil = xr.open_dataset(fpath[0])
print(hil)

asp = xr.open_dataset(fpath[4])
print(asp)

slp = xr.open_dataset(fpath[2])
print(slp)

rou = xr.open_dataset(fpath[1])
print(rou)

sur = xr.open_dataset(fpath[3])
print(sur)

<xarray.Dataset>
Dimensions:              (x: 17569, y: 17002)
Coordinates:
  * x                    (x) float64 2.54e+05 2.54e+05 ... 3.067e+05 3.067e+05
  * y                    (y) float64 4.179e+06 4.179e+06 ... 4.23e+06 4.23e+06
Data variables:
    Band1                (y, x) float32 ...
    transverse_mercator  |S1 ...
Attributes:
    Conventions:  CF-1.5
    GDAL:         GDAL 2.2.3, released 2017/11/20
    history:      Thu Oct 03 11:41:53 2019: GDAL Create( hillshade.nc, ... )
<xarray.Dataset>
Dimensions:              (x: 17569, y: 17002)
Coordinates:
  * x                    (x) float64 2.54e+05 2.54e+05 ... 3.067e+05 3.067e+05
  * y                    (y) float64 4.179e+06 4.179e+06 ... 4.23e+06 4.23e+06
Data variables:
    aspect               (y, x) float32 ...
    transverse_mercator  |S1 ...
Attributes:
    Conventions:  CF-1.5
    GDAL:         GDAL 2.2.3, released 2017/11/20
    history:      Fri Oct  4 11:46:25 2019: ncrename -v Band1,aspect aspect.n...
    NCO:      